In [1]:
# скачиваем файл
#№!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl' -O heart.csv

In [2]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
#!pip install scikit-learn==0.22.1

In [3]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/ronitf/heart-disease-uci). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [4]:
data = pd.read_csv('heart.csv')
display(data.shape)
data.head()

(303, 14)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [5]:
#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

1. Обучите решающее дерево из `sklearn` (`sklearn.tree.DecisionTreeClassifier`) без ограничения на максимальную глубину (`max_depth=None`). В качестве сида поставьте `random_state=13`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

   Найдите долю правильных ответов полученного алгоритма на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [7]:
#from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier( #минимальное число объектов в листе
                              max_depth=None, random_state=13)
clf=tree.fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred_train)

1.0

2. Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.
accuracy_score(y_train, y_pred)
    Заметно ли переобучение?

In [8]:
y_pred_test = clf.predict(X_test)
accuracy_score(y_test, y_pred_test)

0.6710526315789473

3. Подберите с помощью кросс-валидации оптимальные гиперпараметры алгоритма. Выбирайте из следующих наборов:


- `max_depth`: [3, 4, 5, 6, 7, 8, 9, 10, None]
- `max_features`: ['auto', 'log2', None]
- `min_samples_leaf`: range(1, 10)
- `min_samples_split`: range(2, 10)
- `criterion`: ['gini', 'entropy']

    В этом вам поможет метод `sklearn.model_selection.GridSearchCV`. Подробнее о том, какие параметры и методы в нем используются, и о примерах работы с ним можно прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
    
    1) Создайте решающее дерево - не забудьте поставить `random_state=13`.
    
    2) Задайте `param_grid` - сетку (словарь) гиперпараметров и их значений, по которой будет проходить метод.
    
    3) Вызовите метод `GridSearchCV` - в качестве параметра `estimator` задайте решающее дерево из первого шага, `param_grid` - сетку из второго. Задайте параметр `cv=5`, чтобы кросс-валидация проходила по 5 фолдам. Также задайте параметр `scoring='accuracy'`, чтобы оценка качества моделей на кросс-валидации проходила с помощью доли правильных ответов. Запустите метод на обучающей выборке с помощью `fit`. 
    
    4) Выведите лучшие параметры с помощью атрибута `best_params_`.
    
    Какое значение глубины дерева получилось оптимальным?

In [9]:
parametrs={'criterion':['gini', 'entropy']
           , 'max_depth':  [3, 4, 5, 6, 7, 8, 9, 10, None]
           , 'max_features': ['auto', 'log2', None]
           , 'min_samples_leaf': range(1, 10)
           , 'min_samples_split': range(2, 10)
           #, 'min_samples_leaf': [2]
          }

Определим наилучшее количество фолдов.  

In [47]:
cross_val_score(clf, X_train, y_train, cv=3).mean()

0.7665497076023392

In [46]:
cross_val_score(clf, X_train, y_train, cv=4).mean()

0.7798402255639099

In [43]:
cross_val_score(clf, X_train, y_train, cv=5).mean()

0.7792270531400967

In [44]:
cross_val_score(clf, X_train, y_train, cv=6).mean()

0.7798719772403983

In [45]:
cross_val_score(clf, X_train, y_train, cv=7).mean()

0.7621753246753247

In [48]:
%%time
#from sklearn import tree
#from sklearn.model_selection import GridSearchCV

clf=tree.DecisionTreeClassifier(random_state=13)
grid_search_cv_clf=GridSearchCV(clf, parametrs, cv=4
                                #, n_jobs=-1
                               )
grid_search_cv_clf.fit(X_train, y_train)


Wall time: 1min 33s


GridSearchCV(cv=4, estimator=DecisionTreeClassifier(random_state=13),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
                         'max_features': ['auto', 'log2', None],
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10)})

In [49]:
grid_search_cv_clf.best_params_

{'criterion': 'entropy',
 'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 7}

In [50]:
#from sklearn.model_selection import GridSearchCV

4. Какое лучшее усредненное значение доли правильных ответов получилось на кросс-валидации (для оптимальных значений гиперпараметров)? Вам поможет атрибут `best_score_`. Ответ округлите до двух знаков после запятой и дайте в процентах.

In [51]:
grid_search_cv_clf.best_score_

0.8504072681704261

5. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

In [52]:
best_clf=grid_search_cv_clf.best_estimator_
y_pred_train= best_clf.predict(X_train)

In [53]:
best_clf.score(X_train, y_train)

0.9074889867841409

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_train, y_train)

0.9074889867841409

6. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Уменьшилось ли переобучение?

In [55]:
y_pred= best_clf.predict(X_test)
accuracy_score(y_pred, y_test)

0.7236842105263158

7. Решающее дерево позволяет предсказывать не только классы, но и вероятности классов - с помощью метода `predict_proba`. Посмотрите на вероятности классов полученного решающего дерева и посчитайте значение AUC-ROC. Ответ округлите до двух знаков после запятой.

In [56]:
from sklearn.metrics import roc_auc_score

y_pred_proba=best_clf.predict_proba(X_test)
roc_auc_score(y_test, y_pred_proba[:, 1])


0.7758985200845666

8. Какой признак является самым важным по мнению полученного решающего дерева? Чтобы это проверить, вам поможет атрибут `feature_importances_`.

In [57]:
best_clf.feature_importances_

array([0.07912893, 0.00936149, 0.05220983, 0.04252294, 0.07523743,
       0.        , 0.02613543, 0.08348782, 0.13713699, 0.07881687,
       0.17916423, 0.08695546, 0.14984258])

In [58]:
pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_clf.feature_importances_
}).sort_values(by='importance', ascending=False).reset_index(drop=True)

,feature,importance
0,slope,0.179164
1,thal,0.149843
2,exang,0.137137
3,ca,0.086955
4,thalach,0.083488
5,age,0.079129
6,oldpeak,0.078817
7,chol,0.075237
8,cp,0.052210
9,trestbps,0.042523
